In [ ]:
# Just for the notebooks, the cwd needs to be set to the root of the project
import os
from pathlib import Path
home = Path.home()

cwd = os.getcwd()
if not 'initial_cwd' in locals():
	initial_cwd = cwd

# check if any of the parent directories is 'notebooks'
relative_path = Path(cwd).relative_to(home)
if 'notebooks' in relative_path.parts:
	# if so, change the current working directory to the root of the project
	while relative_path.parts and not 'notebooks' in os.listdir(cwd):
		cwd = os.path.dirname(cwd)
		relative_path = Path(cwd).relative_to(home)
	if 'notebooks' in os.listdir(cwd):
		os.chdir(cwd)

print(f"Current working directory: {os.getcwd()}")
print(f"Initial working directory: {initial_cwd}")

In [ ]:
%pip install -r requirements.txt --quiet
from dotenv import load_dotenv
load_dotenv()


# Setup logging

In [ ]:

import logging
from LoggingColor import ColorHandler
import sys
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s.%(msecs)03d %(name)-16s [%(levelname)-1s]: %(message)s',
    handlers=[
        logging.FileHandler(f'log.log',mode='w'),
        #logging.StreamHandler(sys.stdout),
        ColorHandler(sys.stdout)
    ],
    datefmt='%H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.debug("Debug log test")
logger.info("Info log test")
logger.warning("Warning log test")
logger.error("Error log test")
logger.critical("Critical log test")

# Fetch magisk releases

In [ ]:
from dataclasses import dataclass, field

@dataclass
class MagiskRelease:
    tag_name: str
    release_name: str
    prerelease: bool
    debug: bool
    filename: str
    url: str

    def download(self, download_dir: str, filename: str = None, overwrite: bool = False) -> str:
        import requests
        import os
        from tqdm.auto import tqdm

        if filename is None:
            filename = self.filename

        os.makedirs(download_dir, exist_ok=True)
        out_path = os.path.join(download_dir, filename)

        if not overwrite and os.path.exists(out_path):
            logger.warning(f"File {out_path} already exists, skipping download.")
            return out_path

        logger.info(f"Downloading magisk {self.tag_name} to {out_path}")
        temp_path = out_path + ".part"
        with requests.get(self.url, stream=True) as r:
            total_size = int(r.headers.get('content-length', 0))
            with tqdm(total=total_size, unit='B', unit_scale=True, desc=filename) as pbar:
                with open(temp_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
                        pbar.update(len(chunk))
            assert total_size == os.path.getsize(temp_path), f"Downloaded file size {os.path.getsize(temp_path)} does not match expected size {total_size}"

        os.rename(temp_path, out_path)

        return out_path

def fetchMagiskReleases() -> list[MagiskRelease]:
    import requests
    import re

    with requests.Session() as s:
        headers = {
            'X-GitHub-Api-Version': '2022-11-28',
            'Accept': 'application/vnd.github+json',
        }

        res = s.get("https://api.github.com/repos/topjohnwu/Magisk/releases", headers=headers)

    assert res.status_code == 200, f"Failed to fetch OTA page: {res.status_code}"

    releases = res.json()
    logger.info(f"Found {len(releases)} releases")
    available_releases = []
    for release in releases:
        tag_name = release['tag_name']
        release_name = release['name']
        prerelease = release['prerelease']
        for asset in release['assets']:
            filename = asset['name']

            url = asset['browser_download_url']
            if not filename.endswith('.apk'):
                logger.debug(f"Skipping non-apk asset: {filename}")
                continue
            
            # remove some dud file names
            if filename.startswith('stub'):
                logger.debug(f"Skipping stub asset: {filename}")
                continue

            # check if the asset is a debug build
            debug = 'debug' in filename.lower() or 'dbg' in filename.lower()

            release_info = MagiskRelease(
                tag_name=tag_name,
                release_name=release_name,
                prerelease=prerelease,
                debug=debug,
                filename=filename,
                url=url
            )
            available_releases.append(release_info)
            logger.info(f"Found Magisk release: {release_info}")

    return available_releases

magisk_releases = fetchMagiskReleases()

In [ ]:
import pandas as pd
magisk_releases_df = pd.DataFrame(magisk_releases)
magisk_releases_df

magisk_version = "29.0"

matching_releases = [r for r in magisk_releases if r.tag_name == magisk_version or r.tag_name == f"v{magisk_version}"]

# i don't want debug builds
matching_releases = [r for r in matching_releases if not r.debug]

assert len(matching_releases) > 0, f"No matching releases found for version {magisk_version}"
assert len(matching_releases) == 1, f"Multiple matching releases found for version {magisk_version}: {matching_releases}"

matching_release = matching_releases[0]
download_dir = os.path.join(os.getcwd(), 'downloads')
matching_release.download(download_dir, overwrite=False)